In [1]:
!nvidia-smi

Thu Aug 10 18:23:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.191.01   Driver Version: 450.191.01   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   29C    P0    59W / 300W |  31022MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import Dataset, DatasetDict

wow = DatasetDict.load_from_disk("wow")

In [7]:
model_ckpt = "t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_ckpt)
wizard = "Wizard:"
apprentice = "Apprentice:"
nok = "<nok>"
tokenizer.add_tokens([wizard, apprentice, nok])

/raid/kavin-intern-maunendra/anaconda3/envs/wow/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


3

In [8]:
import yake

language = "en"
max_ngram_size = 1
deduplication_thresold = 0.9
deduplication_algo = 'seqm'
windowSize = 1
numOfKeywords = 10

custom_kw_extractor_1 = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, dedupFunc=deduplication_algo, windowsSize=windowSize, top=numOfKeywords, features=None)


In [9]:
import spacy
def masker(inp, mask, ref_in, ref_out):
    op = inp
    msk = []
    j = 0
    for i, w in enumerate(mask):
        if w.lower() in ref_out.lower() and w.lower() not in ref_in.lower():
            op = op.replace(w, f"<extra_id_{j}>")
            j+=1
            msk.append(w)


    return op, " ".join(msk)

en = spacy.load('en_core_web_sm')
def key_exrt(k_lst, strng):

    keywords = []
    k2 = []
    if len(k_lst) == 0:
        keywords += [token for token in en(strng) if
              not token.is_punct
              and not token.is_currency
              and not token.is_digit
              and not token.is_oov
              and not token.is_space
              and not token.is_stop
              and not token.like_num
              and not token.pos_ in []]
    else:
        keywords += [key for key, _ in k_lst]
        k2 += [key for key, _ in k_lst]

    return keywords, ("; ".join(keywords) + ";"), k2

In [10]:
import torch
def prep1(df):
    out = {"labels": [], "attention_mask": [], "input_ids": []}
    for i, _ in enumerate(df["gold_pass"]):
        nok_tk = "" 
        if df["gold_pass"][i] == float("-inf"):
            nok_tk = "<nok> "
        keys = custom_kw_extractor_1.extract_keywords(df["response"][i])
        key, strng, js_ks = key_exrt(keys, df["response"][i])
        pas = df["all_pass"][i][int(df["gold_pass"][i])] if df["gold_pass"][i] != float("-inf") else ""
        msk_res, gpe_out = masker(df["response"][i], js_ks, df["context_eou"][i].replace(" <eou>", ""), pas)
        out_q =  tokenizer(nok_tk + msk_res)
        in_c = tokenizer(df["context"][i])
        out["input_ids"].append(in_c["input_ids"][-256:])
        out["attention_mask"].append(in_c["attention_mask"][-256:])
        out["labels"].append(out_q["input_ids"])

    return out

In [11]:
tokenized_wow = wow.map(prep1, batched=True)

Map:   0%|          | 0/74092 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (552 > 512). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/3939 [00:00<?, ? examples/s]

Map:   0%|          | 0/3865 [00:00<?, ? examples/s]

In [12]:
tokenized_wow.save_to_disk("wow_qg_large")

Saving the dataset (0/2 shards):   0%|          | 0/74092 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3939 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3865 [00:00<?, ? examples/s]

In [10]:
tokenized_wow.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [12]:
model = T5ForConditionalGeneration.from_pretrained(model_ckpt)

In [13]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # # Rouge expects a newline after each sentence
    # decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
    #                   for pred in decoded_preds]
    # decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
    #                   for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [15]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import datasets

batch_size = 16
model_dir = f"wow_qg_{model_ckpt}"

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=70,
    logging_strategy="steps",
    logging_steps=70,
    save_strategy="steps",
    save_steps=140,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    load_best_model_at_end=True,
    report_to="tensorboard",
    gradient_accumulation_steps=2,
    predict_with_generate=True,
    metric_for_best_model="rouge1",
)

data_collator = DataCollatorForSeq2Seq(tokenizer)

metric = datasets.load_metric("rouge")

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized_wow["train"],
    eval_dataset=tokenized_wow["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


In [16]:
if __name__ == "__main__":
    trainer.train()
    trainer.save_model(f"{model_dir}/final")

/raid/kavin-intern-maunendra/anaconda3/envs/wow/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [42]:
tk = [0]*212
len(tk[-256:])

212